<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/embeddings_kinopoisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#! pip install tensorflow_text
#! pip install textattack['tensorflow']
#! pip install datasets --upgrade
#! pip install captum

In [ ]:
# При установке из репозитория при работе не подцепляются другие модели, работает только дефолтный bert-base-uncased

! pip install tensorflow_text
! git clone https://github.com/zloelias/TextAttack.git
! pip install /content/TextAttack[tensorflow] --use-feature=in-tree-build
! pip install datasets --upgrade
! pip install captum

fatal: destination path 'TextAttack' already exists and is not an empty directory.
Processing ./TextAttack
  Created wheel for textattack: filename=textattack-0.3.4-py3-none-any.whl size=377434 sha256=099514435cd03676945fb2aa2cedc867b93d0bff9b160e87d8a059441b20f2b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-6schavlv/wheels/00/07/19/705a89c140a25ea4a53610153f7b1440b6c62115ea7904bdc6
Successfully built textattack
  Attempting uninstall: textattack
    Found existing installation: textattack 0.3.4
    Uninstalling textattack-0.3.4:
      Successfully uninstalled textattack-0.3.4


In [ ]:
import torch
from copy import deepcopy

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
#from captum.attr import visualization as viz

import nltk
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [ ]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder, MultilingualUniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding
from textattack.shared import GensimWordEmbedding, WordEmbedding

from textattack.shared.utils import LazyLoader

from textattack.constraints.pre_transformation.max_word_index_modification import MaxWordIndexModification
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate

from textattack.attack_recipes.attack_recipe import AttackRecipe


In [ ]:
! wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
! wget http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
! tar xzf araneum_none_fasttextcbow_300_5_2018.tgz

--2021-12-14 20:23:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.137.104
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.137.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  35.3MB/s    in 46s     

2021-12-14 20:24:31 (34.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]

--2021-12-14 20:24:31--  http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018

In [ ]:
class TextFoolerJin2019RU(AttackRecipe):
    """Jin, D., Jin, Z., Zhou, J.T., & Szolovits, P. (2019).

    Is BERT Really Robust? Natural Language Attack on Text Classification and Entailment.

    https://arxiv.org/abs/1907.11932
    """

    @staticmethod
    def build(model_wrapper, max_rate, max_seq_len, language):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        #
        gensim = LazyLoader("gensim", globals(), "gensim")

        if language == 'russian':
            keyed_vectors = (
                gensim.models.KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')
            )
        else:
            keyed_vectors = (
                gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format(
                    'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',
                    binary=True
                )
            )
        word_embedding = GensimWordEmbedding(keyed_vectors)
        transformation = WordSwapEmbedding(max_candidates=20, embedding=word_embedding)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(language=language)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        if language == 'english':
            use_constraint = UniversalSentenceEncoder(
                threshold=0.840845057,
                metric="angular",
                compare_against_original=False,
                window_size=15,
                skip_text_shorter_than_window=True,
            )
        else:
            use_constraint = MultilingualUniversalSentenceEncoder(
                threshold=0.840845057,
                metric="angular",
                compare_against_original=False,
                window_size=15,
                skip_text_shorter_than_window=True,
            )

        constraints.append(use_constraint)

        constraints.append(MaxWordIndexModification(max_seq_len))
        constraints.append(MaxModificationRate(max_rate, min_threshold=1))

        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [ ]:
import textattack
import random
import numpy as np

experiments = {
    'word_embedding_swap': [
        {
            'dataset': {'name': 'kinopoisk', 'url': 'zloelias/kinopoisk-reviews'},
            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf'),
            'language': ('ru', 'rus', 'russian'),
            'max_rate': np.linspace(0.1, 0.9, 9),
            'max_seq_len': 312,
            'columns': (['text'], 'labels'),
        },
#        {
#            'dataset': {'name': 'lenta', 'url': 'zloelias/lenta-ru'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-lenta-ru-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
#        {
#            'dataset': {'name': 'ag_news', 'url': 'ag_news'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-ag-news'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
#        {
#            'dataset': {'name': 'rotten_tomatoes', 'url': 'rotten_tomatoes'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-rotten-tomatoes'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
    ]
}

for attack_name in experiments.keys():
    for attack_data in experiments[attack_name]:
        dataset_name, dataset_url = [attack_data['dataset'][k] for k in attack_data['dataset'].keys()]
        dataset_columns = attack_data['columns']
        dataset = HuggingFaceDataset(dataset_url, None, 'test', dataset_columns=dataset_columns)

        base_model_name, model_name = attack_data['model']
        original_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = HuggingFaceModelWrapper(original_model,original_tokenizer)

        la, lan, language = attack_data['language']
        for max_rate in attack_data['max_rate']:
            experiment_result_dir = f'/content/drive/MyDrive/Colab Notebooks/final-project/experiments/{attack_name}/{dataset_name}/{max_rate}/'
            print(experiment_result_dir)
            attack = TextFoolerJin2019RU.build(model, max_rate, attack_data['max_seq_len'], language)

            attack_args = textattack.AttackArgs(
                num_examples=200, 
                log_to_csv=experiment_result_dir + "log.csv", 
                checkpoint_interval=50, 
                checkpoint_dir=experiment_result_dir + "checkpoints", 
                disable_stdout=True
            )
            attacker = textattack.Attacker(attack, dataset, attack_args)
            attacker.attack_dataset()
        #print(attack_data)
        #print(model_name, model_url)

Using custom data configuration zloelias___kinopoisk-reviews-2105e1c5dfdebfd7
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/zloelias___kinopoisk-reviews-2105e1c5dfdebfd7/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset zloelias/kinopoisk-reviews, split test.


/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/kinopoisk/0.1/


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/kinopoisk/0.1/log.csv


...

[Succeeded / Failed / Skipped / Total] 37 / 10 / 3 / 50:  25%|██▌       | 50/200 [11:01<33:05, 13.24s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/kinopoisk/0.9/checkpoints/1639531597387.ta.chkpt" at 2021-12-15 01:26:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 78 / 17 / 5 / 100:  50%|█████     | 100/200 [19:10<19:10, 11.51s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/kinopoisk/0.9/checkpoints/1639532086046.ta.chkpt" at 2021-12-15 01:34:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 119 / 24 / 7 / 150:  75%|███████▌  | 150/200 [28:29<09:29, 11.40s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/kinopoisk/0.9/checkpoints/1639532644914.ta.chkpt" at 2021-12-15 01:44:04 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 159 / 29 / 12 / 200: 100%|██████████| 200/200 [36:16<00:00, 10.88s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/kinopoisk/0.9/checkpoints/1639533111643.ta.chkpt" at 2021-12-15 01:51:51 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 159 / 29 / 12 / 200: 100%|██████████| 200/200 [36:16<00:00, 10.88s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 159    |
| Number of failed attacks:     | 29     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 14.5%  |
| Attack success rate:          | 84.57% |
| Average perturbed word %:     | 5.51%  |
| Average num. words per input: | 320.27 |
| Avg num queries:              | 731.89 |
+-------------------------------+--------+
